In [1]:
# change the directory
import os 
os.chdir("c:\\Users\\jzou2\\Fall 2024-2025\\Project\\OptiMed")
from typing import TypedDict, List, Annotated
from operator import add


# import relevant packages
from agents.base_llm import OpenAILLMs
from agents.master import MasterWorkflow
from agents.user import UserLLM
from agents.diagnosis import DiagnosisLLM

# from agents.diagnosis_team.diagnosis_coordinator import DiagnosticCoordinator
# from agents.diagnosis_team.generalist import GeneralistLLM

In [3]:
class Report(TypedDict):
    """
    Defines the state for the multi-agent workflow
    Includes messages, patient input, and workflow states
    """
    patient_input: str
    messages: Annotated[List[dict], add]
    diagnosis: str
    recommendations: str
    appointment_details: str

mocked_report_empty : Report = {'patient_input': '',
                          'messages' : [],
                          'diagnosis' : '',
                          'recommendations' : '',
                          'appointment_details' : '',
}

mocked_report_patient_filled : Report = {'patient_input': 'The user is a 45-year-old male named John Doe who is experiencing persistent chest pain, shortness of breath, occasional dizziness, and fatigue for the past two weeks. His medical history includes hypertension managed with medication and borderline high cholesterol with no current treatment.',
                          'messages' : [],
                          'diagnosis' : '',
                          'recommendations' : '',
                          'appointment_details' : '',
}

mocked_report_diagnosis_filled : Report = {'patient_input': 'The user is a 45-year-old male named John Doe who is experiencing persistent chest pain, shortness of breath, occasional dizziness, and fatigue for the past two weeks. His medical history includes hypertension managed with medication and borderline high cholesterol with no current treatment.',
                          'messages' : [],
                          'diagnosis' : 'The possible diagnoses for John Doe include acute coronary syndrome, stable angina, heart failure, hypertensive heart disease, pulmonary embolism, gastroesophageal reflux disease, or anxiety/panic attack, and further tests are needed to confirm the diagnosis.',
                          'recommendations' : '',
                          'appointment_details' : '',
}

mock_patient_info = """
John Doe, a 45-year-old male, reports experiencing persistent chest pain, shortness of breath, occasional dizziness, and fatigue for the past two weeks. His medical history includes hypertension diagnosed five years ago, managed with medication, and borderline high cholesterol with no current treatment. He has a family history of heart disease, as his father suffered a heart attack at age 50. He has no known allergies.
"""



#### Test Base LLMs

In [2]:
# PASS
def test_base_llm():
    test_agent = OpenAILLMs()
    print(test_agent.prompt)
    test_agent_result = test_agent("What is the weather today")
    return test_agent_result
print(test_base_llm())

START OF TEMPLATE
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought and Reasoning: you should always think about what to do and reason out whether tool usage is necessary
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought and Reasoning/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}

END OF TEMPLATE
input_variables=[] input_types={} partial_variables={} messages=[SystemMessage(content='You are a helpful AI assistant. When using tools, follow these steps:\n\n1. Understand what is being asked.\n2. Break down complex tasks into smaller steps.\n3. Use available tools when necessary.\n4. Provide clear, concise

ValueError: Prompt missing required variables: {'agent_scratchpad', 'tools', 'tool_names'}

#### Test Master Delegation

In [ ]:
# PASS : USER INTERACTION
def test_master_default():
    master_agent = MasterWorkflow()
    output = master_agent.test_call(mocked_report_empty)
    return output
print(test_master_default())

# PASS : DIAGNOSIS
def test_master_diagnosis():
    master_agent = MasterWorkflow()
    output = master_agent.test_call(mocked_report_patient_filled)
    return output
# print(test_master_delegation())

# PASS : RECOMMENDATION
def test_master_recommendation():
    master_agent = MasterWorkflow()
    output = master_agent.test_call(mocked_report_diagnosis_filled)
    return output
# print(test_master_recommendation())



#### Test User Agent and Tools

In [ ]:
# PASS : Works With 'DONE' and 'Generates Reports'
def test_user():
    "my symptoms are shortness of breath and acute chest pain"
    "32"
    "i do not have any medical history"
    user_agent = UserLLM()
    output = user_agent()
    return output
print(test_user())

#### Test Diagnosis Agent and Tools

In [ ]:
def test_diagnose():
    diagnosis_agent = DiagnosisLLM()
    output = diagnosis_agent(mocked_report_)
    return output
print(test_diagnose())

#### Test Synergies/Workflow

In [ ]:
# STEP ONE PASS : MASTERS TO USER
def master_to_user():
    master_agent = MasterWorkflow()
    output = master_agent.test_run(mocked_report_empty)
    return output
# print(master_to_user())

# STEP TWO PASS: MASTER TO USER TO DIAGNOSIS
def master_to_diagnosis():
    master_agent = MasterWorkflow()
    output = master_agent.test_run(mocked_report_patient_filled)
    return output
# print(master_to_diagnosis())

# STEP THREE : MASTER TO USER TO DIAGNOSIS TO RECOMMENDATION
def master_to_recommendation():
    return None


#### Main Workflow

In [ ]:
# # creates class 
# final_workflow = MasterWorkflow() # called once

# # starts with initial form
# initiate = final_workflow()